In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/data-storm-10/credit_card_default_train.csv
/kaggle/input/data-storm-10/DATA STORM 1.0 - First Round Competition Guidlines.pdf
/kaggle/input/data-storm-10/credit_card_default_test.csv
/kaggle/input/data-storm-10/Credit_card_default - Business Problem - Assessment Criteria - Data Dictionary.xlsx


In [2]:
df = pd.read_csv("/kaggle/input/data-storm-10/credit_card_default_train.csv")

In [3]:
df.head(5)

,Client_ID,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,...,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
0,A2,1M,M,Graduate,Other,31-45,-1,-1,-1,-1,...,39418,162772,-13982,3437,6004,39418,162772,0,538165,0
1,A3,1M,F,High School,Other,Less than 30,0,-1,-1,-1,...,43530,80811,124590,151818,46200,43530,80811,942,33666,0
2,A4,100K,F,High School,Single,31-45,4,3,2,2,...,0,0,0,0,0,0,0,0,0,1
3,A5,200K,F,Graduate,Single,31-45,2,0,0,0,...,97309,100353,102740,3855,3890,3696,4620,4049,3918,1
4,A6,1M,F,Graduate,Other,31-45,2,2,0,0,...,435354,445271,453899,0,20790,16170,17325,16401,17325,0


In [4]:
df.drop(['Client_ID'], 1, inplace=True)
df.fillna(0, inplace=True)

In [5]:
df.sample(5)

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,...,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
4291,1.5M,F,Graduate,Other,31-45,0,0,0,0,0,...,30441,39406,40490,11582,11573,30051,23181,23116,11608,0
18127,100K,M,Other,Other,31-45,0,0,0,0,0,...,60871,58131,58988,3927,3326,3255,2421,2141,2153,0
23989,400K,F,Graduate,Other,31-45,0,0,0,0,0,...,249589,216465,224885,6930,4620,214830,9240,11612,9252,0
6056,400K,M,Other,Other,Less than 30,0,-1,0,-1,-1,...,3749,0,0,10273,2310,3749,0,0,0,0
13334,100K,F,High School,Single,31-45,2,2,2,2,2,...,35639,34380,35791,3003,2310,6237,0,1964,1964,1


In [6]:
df["Balance_Limit_V1"].unique()

array(['1M', '100K', '200K', '400K', ' 500K', '300K', '1.5M', '2.5M'],
      dtype=object)

In [7]:
df["EDUCATION_STATUS"].unique()

array(['Graduate', 'High School', 'Other'], dtype=object)

In [8]:
df["MARITAL_STATUS"].unique()

array(['Other', 'Single'], dtype=object)

In [9]:
df["AGE"].unique()

array(['31-45', 'Less than 30', '46-65', 'More than 65'], dtype=object)

In [10]:
def map_age(age):
    if age == "Less than 30":
        return 0
    elif age == "31-45":
        return 1
    elif age == "46-65":
        return 2
    else:
        return 3

def map_balance(balance):
    if balance == "100K":
        return 1
    elif balance == "200K":
        return 2
    elif balance == "300K":
        return 3
    elif balance == "400K":
        return 4
    elif balance == "500K":
        return 5
    elif balance == "1M":
        return 10
    elif balance == "1.5M":
        return 15
    else:
        return 20


def map_education(education):
    if education == "Other":
        return 0
    elif education == "High School":
        return 1
    elif education == "Graduate":
        return 2

def map_gender(gender):
    if gender == "M":
        return 0
    return 1

def map_marital(marital):
    if marital == "Single":
        return 1
    return 0

In [11]:
df['Balance_Limit_V1'] = df.Balance_Limit_V1.apply(map_balance)
df['EDUCATION_STATUS'] = df.EDUCATION_STATUS.apply(map_education)
df['AGE'] = df.AGE.apply(map_age)
df['Gender'] = df.Gender.apply(map_gender)
df['MARITAL_STATUS'] = df.MARITAL_STATUS.apply(map_marital)

In [12]:
df.sample(10)

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,...,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
13969,20,0,0,0,0,0,0,0,0,0,...,315955,309397,308900,13860,16170,11550,11550,11550,10808,0
13059,20,0,0,1,1,0,0,0,0,0,...,356756,359418,268535,16170,16170,23112,14306,9774,11550,0
20082,10,1,2,1,2,0,0,0,0,0,...,353181,346618,337923,23160,23114,16170,11550,12012,12705,0
2973,1,0,0,0,1,0,-1,-1,0,-1,...,554,1594,347,2913,1455,0,1940,0,1802,0
10197,10,0,0,1,0,-1,-1,-1,-1,0,...,13391,3562,2901,4659,5720,13391,0,2901,53742,0
22349,3,0,0,0,0,0,0,0,0,0,...,232072,231878,234338,10395,10395,10395,8547,8498,9471,0
13766,3,0,2,0,0,0,0,0,0,0,...,258690,260226,264118,9240,9240,23100,9240,11550,9702,0
22509,3,0,0,1,1,1,-2,-2,-2,-1,...,-3511,1802,0,0,0,0,5313,0,0,0
4006,1,1,2,0,0,-2,-2,-2,-2,-2,...,1141,5276,1132,0,0,1143,5290,1134,3915,0
21318,3,1,2,0,0,0,0,0,0,0,...,110286,105461,61273,5033,5061,3523,42125,3465,4620,0


In [13]:
from sklearn import preprocessing
X = np.array(df.drop(['NEXT_MONTH_DEFAULT'], 1).astype(float))
X = preprocessing.scale(X)
y = np.array(df['NEXT_MONTH_DEFAULT'])

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)

In [15]:
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

In [16]:
param = {
    'eta': 0.31, 
    'max_depth': 5,  
    'objective': 'binary:hinge',
} 
steps = 30

In [17]:
model = xgb.train(param, D_train, steps)

In [18]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

best_preds = model.predict(D_test)
# print(best_preds)
# best_preds = np.asarray([np.around(line) for line in preds])

print("Precision =", precision_score(Y_test, best_preds, average='micro'))
print("Recall =", recall_score(Y_test, best_preds, average='micro'))
print("F1 Score =", f1_score(Y_test, best_preds, average='micro'))
print("Accuracy =", accuracy_score(Y_test, best_preds))

Precision = 0.82875
Recall = 0.82875
F1 Score = 0.82875
Accuracy = 0.82875


Precision = 0.8183333333333334
Recall = 0.8183333333333334
F1 Score = 0.8183333333333332
Accuracy = 0.8183333333333334

In [19]:
# import random
# data = {"f1": 0, "data": []}
# def run(eta, max_depth, steps):
#     model = xgb.train({'eta': eta,'max_depth': max_depth, 'objective': 'binary:hinge'} , D_train, steps)
#     best_preds = model.predict(D_test)
#     f1 = f1_score(Y_test, best_preds, average='micro')
#     if data["f1"] < f1:
#         data["f1"] = f1
#         data["data"] = [eta, max_depth, steps]
# #     print("eta=",eta, "max_depth=", max_depth, "steps=", steps, "F1 Score=", f1)

# for _ in range(1000):
#     run(round(random.uniform(0,1), 2), random.randint(1, 10), random.randint(15, 35))

# print("Best Combination", data)

Best Combination {'f1': 0.8183333333333332, 'data': [0.31, 5, 30]}

In [20]:
test_dataset = pd.read_csv("/kaggle/input/data-storm-10/credit_card_default_test.csv")
test_dataset['Balance_Limit_V1'] = test_dataset.Balance_Limit_V1.apply(map_balance)
test_dataset['EDUCATION_STATUS'] = test_dataset.EDUCATION_STATUS.apply(map_education)
test_dataset['AGE'] = test_dataset.AGE.apply(map_age)
test_dataset['Gender'] = test_dataset.Gender.apply(map_gender)
test_dataset['MARITAL_STATUS'] = test_dataset.MARITAL_STATUS.apply(map_marital)

In [21]:
X_ = np.array(test_dataset.drop(['Client_ID'], 1).astype(float))
X_ = preprocessing.scale(X_)
D_predict = xgb.DMatrix(X_)
best_preds = model.predict(D_predict)

In [22]:
len(test_dataset['Client_ID']) == len(best_preds)

True

In [23]:
with open("output.csv", "w") as f:
    f.write("Client_ID,NEXT_MONTH_DEFAULT\n")
    for i in range(len(best_preds)):
        f.write("{},{}\n".format(test_dataset['Client_ID'][i], int(best_preds[i])))